# HW 2 BCH394P
## Jacob Stambaugh, JS88749
02/25/21 | Due: 03/09/21

# Problem 1: Hidden Markov Model for finding transmembrane protein sequences
Proteins in a cell are often synthesized by a ribosome at one location in a cell (for example, in the
cytosol or attached to the endoplasmic reticulum), then shipped someplace else in the cell where the
proteins actually do their work. About a fourth to a third of the proteins get attached to membranes,
often with a portion of the protein on one side of a membrane, a portion penetrating the membrane, and
a portion on the other side of the membrane:

( pic 1)

In this problem, we’ll train a hidden Markov model to detect the regions of these so-called “integral
membrane proteins” that end up buried in the membrane. The reason we can do this is because the
amino acids in these regions tend to be much greasier in character than the amino acids in the nonmembrane-associated part of the protein.
We’ll construct a very simple hidden Markov model of the following architecture:

(pic2)

## Part 1, Problem 1
Calculate the frequencies of amino acids in the first two files (remember your programs from the first
problem set?). Use these frequencies to generate the emission probability tables of the S and T states of
the HMM.

In [12]:
# From Before: find freq of each base
def calc_freq_single(txt):
    # Define dictionary
    aa_emission={}
    le=0
    
    # Loop through each line in text file 
    for line in txt.readlines():
        line=line.strip('\r \n')
        # loop through each line to update dict nucleotides
        for ammino in line:
            le+=1
            if aa_emission.get(ammino)!=None:
                aa_emission.update({ammino:aa_emission.get(ammino)+1})
            else:
                aa_emission.update({ammino:1})
                
    for i in aa_emission.keys():
        aa_emission.update({i:aa_emission.get(i)/le})
    
    return aa_emission


In [2]:
# Calc frequencies - Transmembrane
trans_mem = open('transmembrane_sequences.txt')
transmem_freq=calc_freq_single(trans_mem)

# Display Results
print('Emission probabilities for Transmembrane protiens:')
for i in transmem_freq.keys():
    print('Ammino Acid:',i,'and it''s probability',round(transmem_freq.get(i),4))

Emission probabilities for Transmembrane protiens:
Ammino Acid: W and its probability 0.0226
Ammino Acid: L and its probability 0.158
Ammino Acid: N and its probability 0.0209
Ammino Acid: M and its probability 0.0316
Ammino Acid: V and its probability 0.0982
Ammino Acid: C and its probability 0.0265
Ammino Acid: G and its probability 0.0763
Ammino Acid: P and its probability 0.0288
Ammino Acid: I and its probability 0.1288
Ammino Acid: Y and its probability 0.0414
Ammino Acid: F and its probability 0.1072
Ammino Acid: A and its probability 0.0837
Ammino Acid: S and its probability 0.068
Ammino Acid: T and its probability 0.0523
Ammino Acid: R and its probability 0.0059
Ammino Acid: Q and its probability 0.0139
Ammino Acid: H and its probability 0.0077
Ammino Acid: E and its probability 0.0095
Ammino Acid: D and its probability 0.0091
Ammino Acid: K and its probability 0.0092


In [3]:
# Calc frequencies - soluble
soluble = open('soluble_sequences.txt')
soluble_freq=calc_freq_single(soluble)

# Display Results
print('Emission probabilities for Soluble protiens:')
for i in soluble_freq.keys():
    print('Ammino Acid:',i,'and it''s probability',round(soluble_freq.get(i),4))

Emission probabilities for Soluble protiens:
Ammino Acid: M and its probability 0.0213
Ammino Acid: P and its probability 0.0433
Ammino Acid: T and its probability 0.0614
Ammino Acid: S and its probability 0.0956
Ammino Acid: G and its probability 0.0512
Ammino Acid: I and its probability 0.0586
Ammino Acid: E and its probability 0.0634
Ammino Acid: L and its probability 0.0889
Ammino Acid: D and its probability 0.0581
Ammino Acid: Q and its probability 0.0371
Ammino Acid: F and its probability 0.0465
Ammino Acid: K and its probability 0.0715
Ammino Acid: A and its probability 0.0536
Ammino Acid: V and its probability 0.0526
Ammino Acid: N and its probability 0.0594
Ammino Acid: R and its probability 0.05
Ammino Acid: H and its probability 0.0227
Ammino Acid: W and its probability 0.0137
Ammino Acid: Y and its probability 0.0392
Ammino Acid: C and its probability 0.0121


## Explanation for Problem 1: part 1

Looking

## Part 2, Problem 1
Calculate the frequencies of beginning a sequence in each of the 2 states in the third file. Use these to
calculate the transition probabilities from the beginning state of the HMM.

In [4]:
# Part 2: Calculate freq of beginning in each state

# load data
state_dict={'S':0,'T':0}
state_seq = open('state_sequences.txt')

num_line=0

# loop and count S&T's in the begining of each line
for line in state_seq.readlines():
    line=line.strip('\r\n')
    state_dict.update({line[0]:state_dict.get(line[0])+1})
    num_line+=1
    
# Divide dict by length to find probability
for key in state_dict.keys():
    state_dict.update({key:state_dict.get(key)/num_line})

In [5]:
# Print Results
for key in state_dict.keys():
    print('Domain %s occurs at the beginning of read %.3f %% of the time!'%(key,state_dict.get(key)*100))

Domain S occurs at the beginning of read 98.536 % of the time!
Domain T occurs at the beginning of read 1.464 % of the time!


## Part 3, Problem 1
Calculate the number of occurrences of each digram in the state file. (e.g., SS or TT or ST or TS. A
digram is a 2 character string. In general, a string of n characters is an n-gram.) You can use your other
program from problem set#1 for this. From the numbers of observations, you should be able to
construct the transition probability matrix for the HMM.

In [13]:
# From Before: Calc frequencies function

def calc_freq(txt):
    # Define dict for di-nucleotides
    difreq_dict={}
    prev_base=-1
    
    # Loop through each line in text file
    for line in txt.readlines():
        # SKip emtpy lines
        if not line.strip():
            continue
        # get new line
        line=line.strip('\r\n')
        
        # loop through each line to count dinucleotides
        for i in range(0,len(line)-1):
                
            base=line[i:i+2]# gets di-pair
            
            # update dictionary
            if difreq_dict.get(base)!=None:
                difreq_dict.update({base:difreq_dict.get(base)+1})
            else:
                # add new pair
                difreq_dict.update({base:1})
               
            # attach(save) last base pair to new line
            #if i == len(line)-2:
                #prev_base=line[len(line)-1]
                    
    # divide by total length
    le=0
    for i in difreq_dict.keys():
        le+=difreq_dict.get(i)
  
    for i in difreq_dict.keys():
        difreq_dict.update({i:difreq_dict.get(i)})
        
    return difreq_dict

In [14]:
# Part 3: Calculate freq of each digram
import numpy as np

# load data
state_seq = open('state_sequences.txt')

# get freq 
state_seq=calc_freq(state_seq)

# transform into transistion probs
transistion_matrix=np.empty([2,2])

s_state_total=state_seq.get('SS')+state_seq.get('ST')
t_state_total=state_seq.get('TT')+state_seq.get('TS')

# add to matrix: SS,ST,TT,TS
transistion_matrix[0,0]=(state_seq.get('SS')/s_state_total)
transistion_matrix[0,1]=(state_seq.get('ST')/s_state_total)
transistion_matrix[1,0]=(state_seq.get('TT')/t_state_total)
transistion_matrix[1,1]=(state_seq.get('TS')/t_state_total)

# Check matrix
transistion_matrix

array([[0.98721407, 0.01278593],
       [0.95266086, 0.04733914]])

## Part 4, Problem 1
Now, take the amino acid sequence KNSFFFFFFFLIII and calculate the most likely state sequence
using the Viterbi algorithm.

In [108]:
# Part 4: Viterbi Algo

# put emission probs into list for east access
emission=[soluble_freq,transmem_freq]

seq='KNSFFFFFFFLIII'

# Intialize path matrix
path_mat=np.zeros([2,len(seq)]) # 
path_mat[0,0]=1 # begining probability

# intialize pointer matrix
pointer_mat=np.zeros([2,len(seq)])#

# get variables 
b_s=state_dict.get('S')
b_t=state_dict.get('T')

# solve first column
path_mat[0,0]=1*b_s*soluble_freq.get(seq[0])

path_mat[1,0]=1*b_t*transmem_freq.get(seq[0])

# Iterate throguh and solve 

# outer loop - goes through input sequence
for aa in range(1,len(seq)):
    
    # inner loop - solves for both S + T states
    for state in range(0,2):
        temp_list=[]
        # Get emission prob per state
        emi=emission[state].get(seq[aa])
            # inner inner loop - goes through every previous value
        for prev_state in range (0,2):
            # prev state probability 
            prev_state_prob=path_mat[prev_state,aa-1]
            
            # gett transmission prob (per state)
            trans=transistion_matrix[prev_state,state]
            
            # calculate
            temp_list.append(prev_state_prob*emi*trans)
    
    
        # select max probability and update matrix
        indx=temp_list.index(max(temp_list))
        path_mat[state,aa]=temp_list[indx]
        #path_mat[state,aa]=sum(temp_list)
        
        # update pointer
    
# back track to get path
    

In [99]:
#len(soluble_freq.keys())
#len(transmem_freq.keys())
# Intialize path matrix
path_mat


array([[7.04416691e-02, 4.13762824e-03, 3.92133682e-04, 1.81633614e-05,
        8.58452697e-07, 4.06366978e-08, 1.92385760e-09, 9.10817716e-11,
        4.31211492e-12, 2.04149917e-13, 1.84855369e-14, 1.09586401e-15,
        6.52925863e-17, 3.89163269e-18],
       [1.34812113e-04, 1.89824857e-05, 3.65649669e-06, 5.56240770e-07,
        2.77287096e-08, 1.31784471e-09, 6.24096395e-11, 2.95474793e-12,
        1.39887877e-13, 6.62276006e-15, 4.61987995e-16, 3.32472324e-17,
        2.00668464e-18, 1.19717273e-19]])

In [107]:
pointer_mat

[[], [0, 0], [0, 0]]

In [97]:
a=[1,2,3]
sum(a)

6

## Wrapping Up Problem 1
Turn in the transition probability matrix, emission probability matrix, the Viterbi matrix, and the most
likely state sequence.

## Problem 2
How many transmembrane amino acid segments (i.e., segments of all T’s) does the average integral
membrane protein have? Show your calculation.

In [9]:
# Problem 2 Code:

## Problem 2 Explanation

In

## Problem 3
Draw a reasonable alternate architecture for the transmembrane sequence predicting HMM. Write
why you might prefer one of the two HMM architectures over the other.

In [10]:
# Problem 3 Code:

## Problem 3 Explanation

THis

## Problem 4
Draw a reasonable architecture for an HMM that would identify protein secondary structures (e.g.,
alpha helix, beta strand, random coil) within protein sequences.

In [11]:
# Problem 4 Code:

## Problem 4 Explanation

THis

## Problem 5
Suggest an application (ANY application!) of HMM’s to a biological problem other than the
applications we have discussed (i.e., not gene finding, CpG islands, protein secondary structure or
transmembrane segment prediction, sequence alignment). Draw a potential HMM architecture that
could be applied to that problem.

## Problem 5 Explanation
This